In [1]:
from imageai.Detection import ObjectDetection
import cv2
from uuid import uuid4

In [2]:
detector = ObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath("model/yolo.h5")
detector.loadModel()

2022-11-25 22:09:56.845452: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-11-25 22:09:56.849174: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
img_array = cv2.imread("media/images/4.jpg")

In [9]:
custom = detector.CustomObjects(person=True)

detections = detector.detectObjectsFromImage(
    custom_objects=custom,
    input_image=img_array,
    # output_image_path="media/output_images/image_new.jpg",
    minimum_percentage_probability=30,
    input_type="array",
    output_type="array"
)

In [10]:
detections

(array([[[2, 0, 0],
         [2, 0, 0],
         [2, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[2, 0, 0],
         [2, 0, 0],
         [2, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[2, 0, 0],
         [2, 0, 0],
         [2, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        ...,
 
        [[1, 0, 4],
         [1, 0, 2],
         [1, 0, 4],
         ...,
         [2, 0, 0],
         [2, 0, 0],
         [2, 0, 0]],
 
        [[0, 0, 1],
         [0, 0, 0],
         [0, 0, 1],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]]], dtype=uint8),
 [{'name': 'person',
   'percentage_probability': 99.6704638004303,
   'box_points': [40, 26, 177, 183]}])

In [14]:
len(detections)

2

In [15]:
cv2.imwrite("media/output_images/image_new_array.jpg", detections[0])

True

In [17]:
vid = cv2.VideoCapture("media/input_video/video_2.mp4")

while vid.isOpened():
    # Capture frame-by-frame
    ret, frame = vid.read()
    print(ret)

    if ret:
        cv2.imwrite(f"media/images_from_video/__{uuid4()}.jpg", frame)

    # while not ret:
    #     print("Can't receive frame. Retrying ...")
    #     vid.release()
    #     vid = cv2.VideoCapture("media/input_video/video_2.mp4")
    #     ret, frame = vid.read()

    else:
        continue

    # When everything done, release the video capture object
    vid.release()


True
